# Imports

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
from matplotlib import pyplot as plt
import time
import pandas as pd
import random
from dataclasses import dataclass

# Ball Shuffler

In [35]:
import random

def initial_ball_position(n=3):
    return random.randint(1, n)

initial_position = initial_ball_position()
print(initial_position)

def generate_shuffle_moves(n=3, num_moves=3):
    moves = []
    
    for _ in range(num_moves):
        # Randomly pick two different cups
        cup1, cup2 = random.sample(range(1, n + 1), 2)
        moves.append((cup1, cup2))
    
    return moves

shuffle_moves = generate_shuffle_moves()
print(shuffle_moves)

def final_ball_position(initial_position, shuffle_moves):
    position = initial_position
    for move in shuffle_moves:
        # If the ball's current position matches one of the cups in the move, swap it.
        if position == move[0]:
            position = move[1]
        elif position == move[1]:
            position = move[0]
    
    return position

final_position = final_ball_position(initial_position, shuffle_moves)
print(final_position)

3
[(2, 1), (1, 2), (1, 3)]
1


# Data generator

In [42]:
def generate_cup_shuffling_scenario(n=3, num_moves=3):
    # Generate initial ball position and shuffle moves
    initial_position = initial_ball_position(n)
    shuffle_moves = generate_shuffle_moves(n, num_moves)
    
    # Calculate the final ball position
    final_position = final_ball_position(initial_position, shuffle_moves)
    
    # Construct the input and output strings
    input_str = f"ball in cup {initial_position}\n"
    input_str += "\n".join([f"cup {move[0]} to cup {move[1]}" for move in shuffle_moves])
    
    output_str = f"ball in cup {final_position}"
    
    return input_str, output_str

input_scenario, output_scenario = generate_cup_shuffling_scenario()
print(input_scenario)
print(output_scenario)
input_scenario


ball in cup 1
cup 2 to cup 3
cup 3 to cup 1
cup 3 to cup 1
ball in cup 1


'ball in cup 1\ncup 2 to cup 3\ncup 3 to cup 1\ncup 3 to cup 1'

# Model

In [43]:
# Tokenizer, we want BOS, EOS tokens
tokens = {
    1: "<BOS>",
    2: "<EOS>",
    3: "\n",
    4: "ball",
    5: " in",
    6: " cup",
    7: " to",
    8: " 1",
    9: " 2",
    10: " 3",
    11: " 4",
    12: " 5",
    13: " 6",
    14: " 7",
    15: " 8",
    16: " 9"}

# Create a config file for the cup shuffling task
@dataclass
class MASTER_CONFIG:
    seed: int = 1337
    block_size: int = 12 
    batch_size: int = 32
    training_split: float = 0.8
    d_model: int = 128
    
    n_cups: int = 3
    n_moves: int = 3
    tokens: dict = tokens
    vocab_size: int = len(tokens)
    n_embed: int = 1024
    n_heads: int = 8
    head_size: int = 128 # n_embed/n_heads?
    n_layers: int = 8
    dropout: float = 0.1
    
    
    max_iters = 100
    eval_interval = 10
    learning_rate = 2e-5
    
    # Use CUDA or MPS if available else CPU
    if (torch.cuda.is_available()):
        device = torch.device("cuda")
        print("Using CUDA")
    elif (torch.backends.mps.is_available()):
        device = torch.device("mps")
        print("Using Apple Silicon MPS")
    else:
        device = torch.device("cpu")
        print("Using CPU")

    eval_iters = 50

# Method for generating data and labels for batches.
def generate_batch(data, split, config=MASTER_CONFIG):
    pass

# Dummy model for testing, basic feed-forward neural network with embeddings of dimension d_model. 
class DummyModel(nn.Module):
    def __init__(self, config=MASTER_CONFIG):
        super().__init__()
        self.config = config
        self.embedding = nn.Embedding(config.vocab_size, config.d_model)
        self.linear = nn.Sequential(
            nn.Linear(config.d_model, config.d_model),
            nn.ReLU(),
            nn.Linear(config.d_model, config.vocab_size)
        )
        
        self.num_parameters = sum(p.numel() for p in self.parameters() if p.requires_grad)
        
    def forward(self, x, targets=None):
        x = self.embedding(x)
        x = self.linear(x)
        return x

# Method for evaluating the loss of the PyTorch model on the validation set without defining the model.
@torch.no_grad()
def evaluate_model(model, criterion):
    out = {}
    model.eval()
    
    for split in ["train", "val"]:
        total_loss = 0
        total_tokens = 0
        
        for batch in generate_batch(split=split):
            # Get the inputs and targets
            inputs = batch["inputs"]
            targets = batch["targets"]
            
            # Get the model outputs
            outputs = model(inputs)
            
            # Calculate the loss
            loss = criterion(outputs, targets)
            
            # Update the total loss and tokens
            total_loss += loss.item()
            total_tokens += targets.shape[0] * targets.shape[1]
        
        # Calculate the average loss
        avg_loss = total_loss / total_tokens
        
        # Store the average loss
        out[f"{split}_loss"] = avg_loss
    